In [10]:
# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
# Load data
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:
# Muncul data awal dan akhir
print(df.head())
print(df.tail())

In [ ]:
# Statistik deskriptif dasar
print(df.describe())

# Cek nilai yang hilang
print(df.isnull().sum())

# Visualisasi distribusi data
df.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Filter kolom terkait
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Ganti nama kolom
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Konversi tipe data
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Ganti tipe data 'Indeks Produksi'
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Buat kolom 'bulan_tahun'
data_filtered['bulan_tahun'] = data_filtered['Tanggal'].dt.to_period('M')

# Rekapan Bulanan
monthly_summary = data_filtered.groupby('bulan_tahun').agg({
    'SO': 'sum',
    # Uncomment other columns as needed
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Muncul Rekapan Bulanan
print(monthly_summary)

In [15]:
# Set alpha value for SES
alpha = 0.1

# Initialize columns for SES forecast and error metrics
monthly_summary['SES Forecast'] = np.nan
monthly_summary['Error'] = np.nan
monthly_summary['MAD'] = np.nan
monthly_summary['MSE'] = np.nan
monthly_summary['MAPE'] = np.nan

# Set initial forecast to the first SO value
monthly_summary.loc[0, 'SES Forecast'] = monthly_summary.loc[0, 'SO']

# Apply SES formula and calculate error metrics for each subsequent row
for i in range(1, len(monthly_summary)):
    # SES Forecast
    monthly_summary.loc[i, 'SES Forecast'] = (alpha * monthly_summary.loc[i-1, 'SO']) + ((1 - alpha) * monthly_summary.loc[i-1, 'SES Forecast'])
    
    # Error
    monthly_summary.loc[i, 'Error'] = monthly_summary.loc[i, 'SO'] - monthly_summary.loc[i, 'SES Forecast']
    
    # MAD (Mean Absolute Deviation)
    monthly_summary.loc[i, 'MAD'] = abs(monthly_summary.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    monthly_summary.loc[i, 'MSE'] = monthly_summary.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if monthly_summary.loc[i, 'SO'] != 0:
        monthly_summary.loc[i, 'MAPE'] = (abs(monthly_summary.loc[i, 'Error']) / monthly_summary.loc[i, 'SO']) * 100
    else:
        monthly_summary.loc[i, 'MAPE'] = np.nan

print(monthly_summary)

In [ ]:
# Create dummy data with SO values set to 0 for 12 months into the future
months_to_forecast = 12

# Get the last month from the original data
last_month = monthly_summary['bulan_tahun'].max().to_timestamp()

# Generate dummy months starting from the next month after the last month in the original data
dummy_data = pd.DataFrame({
    'bulan_tahun': pd.date_range(start=last_month + pd.offsets.MonthBegin(1), periods=months_to_forecast, freq='M').to_period('M'),
    'SO': np.zeros(months_to_forecast)
})

# Combine with the original data
extended_data = pd.concat([monthly_summary[['bulan_tahun', 'SO']], dummy_data], ignore_index=True)

# Initialize columns for SES forecast and error metrics in the extended data
extended_data['SES Forecast'] = np.nan
extended_data['Error'] = np.nan
extended_data['MAD'] = np.nan
extended_data['MSE'] = np.nan
extended_data['MAPE'] = np.nan

# Set initial forecast to the first SO value (from original data)
extended_data.loc[0, 'SES Forecast'] = extended_data.loc[0, 'SO']

# Apply SES formula to the extended dataset
for i in range(1, len(extended_data)):
    # SES Forecast
    extended_data.loc[i, 'SES Forecast'] = (alpha * extended_data.loc[i-1, 'SO']) + ((1 - alpha) * extended_data.loc[i-1, 'SES Forecast'])
    
    # Error
    extended_data.loc[i, 'Error'] = extended_data.loc[i, 'SO'] - extended_data.loc[i, 'SES Forecast']
    
    # MAD (Mean Absolute Deviation)
    extended_data.loc[i, 'MAD'] = abs(extended_data.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    extended_data.loc[i, 'MSE'] = extended_data.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if extended_data.loc[i, 'SO'] != 0:
        extended_data.loc[i, 'MAPE'] = (abs(extended_data.loc[i, 'Error']) / extended_data.loc[i, 'SO']) * 100
    else:
        extended_data.loc[i, 'MAPE'] = np.nan

# Show the result
print(extended_data)

In [ ]:
# Optional: Plot the actual vs forecasted values
plt.figure(figsize=(14, 6))
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['SO'], label='Actual SO', marker='o')
plt.plot(extended_data['bulan_tahun'].astype(str), extended_data['SES Forecast'], label='SES Forecast', marker='x')
plt.xlabel('Month-Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs SES Forecast')
plt.legend()
plt.tight_layout()
plt.show()